# GCP Demos

### Capstone Project GCP Location: https://console.cloud.google.com/home/dashboard?project=project2-2019-259821

### Geo info: http://donnees.ville.montreal.qc.ca/dataset/geobase
https://robertorocha.info/how-i-made-the-montreal-street-history-map/

In [71]:
import networkx as nx
import requests
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
with open(r"./geojson_dump.json", 'r') as f:    
    data = f.read()

In [73]:
data_json = json.loads(data)

In [74]:
def coord_distance(e1, e2):
    return ((e2[0] - e1[0])**2 + (e2[1] - e1[1])**2)**0.5

In [75]:
G = nx.Graph()
for feature in data_json['features']:
    for idx, coord in enumerate(feature['geometry']['coordinates']):
        coord = tuple(coord)
        if coord not in G:
            G.add_node(coord, **feature['properties'])
        if idx != 0:
            G.add_edge(last_node, coord, length=coord_distance(coord, last_node))
        last_node = coord

In [77]:
pos = nx.spring_layout(G)
#nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 1)
#nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos, edgelist=G.edges())

MemoryError: 